In [11]:
import os
import re
import cv2
import imutils
import sys
import pytesseract
import image
import numpy as np
import docx
import tensorflow as tf
import skimage
import keras
import PIL
import time
import pyautogui
import pandas as pd
import keras

from matplotlib import pyplot as plt
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from autocorrect import spell
from nltk.corpus import words
from docx import Document
from docx.shared import Pt
from docx.enum.section import WD_ORIENT
from keras.preprocessing.image import load_img ,img_to_array ,array_to_img
from skimage import data, io, filters, color
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from PIL import Image
from numpy import zeros, newaxis
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D



def Read_TrainDATA_Facial_Expression():    

########### #######         ##       # ###      ##      #######         ##   ###########   ##                                      
     #      ##     ##      ####      # ## #     ##      ##     ##      ####       #       ####                        
     #      ##     ##     ##  ##     # ##  #    ##      ##      ##    ##  ##      #      ##  ##                              
     #      #######      ##    ##    # ##   #   ##      ##      ##   ##    ##     #     ##    ##                            
     #      ##   ##     ##########   # ##    #  ##      ##      ##  ##########    #    ##########                                
     #      ##    ##   ##        ##  # ##     # ##      ##     ##  ##        ##   #   ##        ##                              
     #      ##     ## ##          ## # ##      ###      #######   ##          ##  #  ##          ##                       
                                                           
    # Assign Arrays
    ImageArray = []
    EmotionList = []
    ImageArray_for_display = []
    
    # Read Data from Train Data Excel file
    ReadExcel = pd.read_excel(Train_Facial_Expression)
    ImageList = ReadExcel['pixels'].tolist()
    Emotion_No = ReadExcel['emotion'].tolist()
    
    # Convert Numbers to Emotions
    # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
    for Counter, Value in enumerate(Emotion_No):
        if Value   == 0:
            EmotionList.append('Angry')
        elif Value == 1:
            EmotionList.append('Disgust')
        elif Value == 2:
            EmotionList.append('Fear')
        elif Value == 3:
            EmotionList.append('Happy')            
        elif Value == 4:
            EmotionList.append('Sad')            
        elif Value == 5:
            EmotionList.append('Surprise')            
        elif Value == 6:
            EmotionList.append('Neutral')     
            
    for I in ImageList:
        Each_Image = [  int(pixel) for pixel in I.split(' ')  ]         
        ImageArray.append(np.asarray(Each_Image).reshape(48,48,1)) 
        ImageArray_for_display.append(np.asarray(Each_Image).reshape(48,48))
    
    
##    # Excel Summary
##    ReadExcel.info()
       
##    # Display Train Data Facial Expression on Screen
##    for J in range(30):
##        plt.figure(figsize=(1.5,1.5))
##        plt.title(EmotionList[J], fontsize=15)
##        plt.imshow(ImageArray_for_display[J])
        
        
    return ImageArray, Emotion_No, EmotionList 
        

    


        
        
        

        
def Machine_Learning_Model(ImageArray):  

###       ###  #######  #######    ######## #                                                                             
####     #### ##     ## ##     ##  #        #                                                         
##  #   #  ## ##     ## ##      ## #        #                                           
##   # #   ## ##     ## ##      ## ######   #                                                        
##    #    ## ##     ## ##      ## #        #                                                         
##         ## ##     ## ##     ##  #        #                                                              
##         ##  #######  #######    ######## ########                                                                                                    

                                
    ImageArray = np.array(ImageArray)   
##    ImageArray = ImageArray / 255.0   
    x_train, x_test, y_train, y_test = train_test_split(ImageArray, Emotion_No, test_size=Test_Data_Size, shuffle=True)
    x_train, x_test = x_train / 255.0, x_test / 255.0 

#    print(x_train.shape) 
#   Print the Shape/Dimension of train data as (28709, 48, 48, 1) = (samples, rows, columns, channels)
#   Note: x_train is IMAGE & y_train is LABEL
#   Note: x_test  is IMAGE & y_test  is LABEL

################################################# CNN #################################################
# When Filter is 32, there will have 32 different Feature Maps
# The more Feature Maps there are, the picture becomes more abstract
# Max pooling is used to find the outliers, this is when network sees the feature

    # Convolutional Neural Network (CNN)
    model = Sequential()  
    model.add(Conv2D(4, kernel_size=kernelSize, 
              activation=Model_Activation,      
              input_shape=(48,48,1))) 
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    
    model.add(Conv2D(4, kernel_size=kernelSize, 
              activation=Model_Activation
              ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))    

    



    
##    model.add(Conv2D(8, kernel_size=kernelSize, 
##              activation=Model_Activation,
##              padding='same',
##              use_bias=False
##              ))
##    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))    
##    model.add(Dropout(0.2))    
    
#   model.add(Conv2D(4, kernel_size=(3, 3), activation=Model_Activation))    
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
       
################################################# CNN #################################################    
    
    model.add(Flatten())
    model.add(Dense(512, activation=Model_Activation))
    model.add(Dense(1024, activation=Model_Activation))    
    model.add(Dense(7, activation=tf.nn.softmax))

    # Compile Model
    model.compile(optimizer=Optimizer_Option, loss=Loss_Option, metrics=['accuracy'])   
    
    if Weight_Mode == 0:
        model.fit(x_train, y_train, epochs=No_of_Epochs)
        model.save_weights(os.path.join(Current_Dir,'Model_Weights\DigitWeights_Trial.h5'))
    elif Weight_Mode == 1:
        model.load_weights(os.path.join(Current_Dir,Saved_Weight))
    
    # Model Score
    score = model.evaluate(x_test, y_test, verbose=100)
    
#    # Model Summary
#    model.summary()
    
    return score, model        










def Facial_Expression_Prediction(image_file):      

#######  #######    ######## #######    #   ####### ###########                                                 
##     # ##     ##  #        ##     ##  #  ##            #                                 
##     # ##     ##  #        ##      ## # ##             #                      
#######  #######    ######   ##      ## # ##             #                         
##       ##    ##   #        ##      ## # ##             #                                  
##       ##     ##  #        ##     ##  #  ##            #                              
##       ##      ## ######## #######    #   #######      #                                       
        
    
        # Convert image to Numbers
        image_file = np.array(image_file)
        image_file = image_file / 255.0 
 
        # shrink image to 48x48 size. This size must fit Train Data size 
        image_file = skimage.transform.resize(image_file, [48,48], mode='reflect')
        image_file = np.reshape(image_file,(1,48,48,1))         # 4 dimensional image for Tensorflow to process
##        print(image_file.shape)
        
        # Do Prediction
        Prediction = model.predict(image_file)
        
        # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
        if Prediction.argmax()   == 0:
            Emotion = 'Angry'
        elif Prediction.argmax() == 1:
            Emotion = 'Disgust'
        elif Prediction.argmax() == 2:
            Emotion = 'Fear'
        elif Prediction.argmax() == 3:
            Emotion = 'Happy'            
        elif Prediction.argmax() == 4:
            Emotion = 'Sad'            
        elif Prediction.argmax() == 5:
            Emotion = 'Surprise'            
        elif Prediction.argmax() == 6:
            Emotion = 'Neutral'        
            
        return Emotion, Prediction

            
            
            
            
            
            
                  
            
            
def Facial_Expression_Recognition(Test_Facial, Frontal_Face_Detection_XML, Profile_Face_Detection_XML, Eyes_Detection_XML):

#######    ########  ######   #######   ######   ##      # ####### ####### #######  #######  ##      #                     
##     ##  #        ##     # ##     ## ##        ###     #    #       #       #    ##     ## ###     #
##     ##  #        ##       ##     ## ##        #  #    #    #       #       #    ##     ## #  #    #     
#######    ######   ##       ##     ## ##   #### #   #   #    #       #       #    ##     ## #   #   #     
##    ##   #        ##       ##     ## ##     ## #    #  #    #       #       #    ##     ## #    #  #           
##     ##  #        ##     # ##     ## ##     ## #     ###    #       #       #    ##     ## #     ###            
##      ## ########  ######   #######   #######  #      ## #######    #    #######  #######  #      ##                      


    # Angle of face detection
    Frontal_face_cascade = cv2.CascadeClassifier(Frontal_Face_Detection_XML)
    Profile_face_cascade = cv2.CascadeClassifier(Profile_Face_Detection_XML)
    eye_cascade  = cv2.CascadeClassifier(Eyes_Detection_XML)

    # Activate PC Video
    VideoFrame = cv2.VideoCapture(0)
    VideoFrame.set(4,640) # set Width
    VideoFrame.set(5,480) # set Height    
    while(True):

        # Capture frame-by-frame, where the frame is "image_file_for_display"
        ret, image_file_for_display = VideoFrame.read()
        image_file_for_display = cv2.flip(image_file_for_display, 1)
        image_file = cv2.cvtColor(image_file_for_display, cv2.COLOR_BGR2GRAY)
        faces = Frontal_face_cascade.detectMultiScale(image_file_for_display, scaleFactor=1.35, minNeighbors=3, \
                minSize=(15, 15))
        
        for (x,y,w,h) in faces:  
            Extra_Frame = 2
            Emotion, Prediction = Facial_Expression_Prediction(image_file[y+Extra_Frame:y+h+Extra_Frame,x-Extra_Frame:x+w+Extra_Frame])
            
            # Change Colour when Face Expression Changes
            FrameColour = (0,0,255)
            if str(Emotion) == 'Angry':
                FrameColour = (0,0,255) # RED
            elif str(Emotion) == 'Disgust':
                FrameColour = (0,0,255) # RED
            elif str(Emotion) == 'Fear':
                FrameColour = (240,0,255) # PINK
            elif str(Emotion) == 'Happy':
                FrameColour = (0,255,0) # GREEN
            elif str(Emotion) == 'Sad':
                FrameColour = (0,120,255) # ORANGE
            elif str(Emotion) == 'Surprise':
                FrameColour = (0,204,204) # YELLOW
            elif str(Emotion) == 'Neutral':
                FrameColour = (255,0,0) # BLUE
            
            cv2.putText(image_file_for_display, str(Emotion) + " " + str(round(Prediction[0][Prediction.argmax()]*100,1)) + "%", \
                        org=(x,y-10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.0,                       \
                        color=FrameColour, thickness=1                    
                       )            
            cv2.rectangle(image_file_for_display,(x-Extra_Frame,y+Extra_Frame),(x+w+Extra_Frame,y+h+Extra_Frame),FrameColour,2) 
            
            # Display No of Object Captured
            cv2.putText(image_file_for_display, 
                        'Object Captured: '+ str(faces.shape[0]),
                        org=(10,20), 
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, 
                        fontScale=0.6,         
                        color=(255,255,255), 
                        thickness=2                    
                       )              
            
            
        cv2.imshow('Facial Expression Recognizer - Colour', image_file_for_display)
        #cv2.imshow('Facial Expression Recognizer - Non-Colour', image_file)
    
        k = cv2.waitKey(30) & 0xff
        if k == 27: # press 'ESC' to quit
            break
    VideoFrame.release()
    cv2.destroyAllWindows()


        
        
        
        

        
        
        
def Duration_Calculation():        

########### ###### ###       ### #######              
    ##        ##   ####     #### #                 
    ##        ##   ##  #   #  ## #                  
    ##        ##   ##   # #   ## #####                 
    ##        ##   ##    #    ## #                   
    ##        ##   ##         ## #    
    ##      ###### ##         ## ########            
                     
    EndTime = time.time()
    EndDateTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    # Calculate script Runtime
    Duration = EndTime - StartTime
    hours = Duration//3600
    Duration = Duration - 3600*hours
    minutes = Duration//60
    seconds = Duration - 60*minutes
    
    print("Start:", StartDateTime,                                      '\t    Loss:', round(score[0], 3)   )
    print("End  :", EndDateTime,                                        '\tAccuracy:', round(score[1]*100, 2),'%'   )
    print('Runtime: %d Hrs %d Min %d Sec' %(hours,minutes,seconds)) 
    
    
    


        
        
   

        
###     ###      ###      ## ###    ##           
####   ####     ## ##     ## ####   ##                 
## ## ## ##    ##   ##    ## ## ##  ##              
##  ###  ##   #########   ## ##  ## ##                  
##       ##  ##       ##  ## ##   ####                         
##       ## ##         ## ## ##    ###   

                        ########## (CAN EDIT) Switch Mode ##########
# Weight_Mode = 0 << Train Model with epochs & Save Weight
# Weight_Mode = 1 << Load Saved Weight to Model. Remember to update Saved_Weight
Weight_Mode = 1
Saved_Weight = 'Model_Weights\DigitWeights_Good_1.h5'


                        ########## (CAN EDIT) Set Parameters for Model ##########
Test_Data_Size = 0.15                               #Size of Test data, 0.2 means 20% of total Train data are Test data
No_of_Epochs = 35
kernelSize=(6,6)                                   #Size of each kernel mapping window
Model_Activation = tf.nn.swish                      #'tf.nn.swish','tf.nn.relu'
Optimizer_Option = 'Adadelta'                          #'Adam','Adamax','Adagrad','Adadelta','Nadam','SGD','RMSprop'
Loss_Option = 'sparse_categorical_crossentropy'    #'sparse_categorical_crossentropy'
    

                        ########## (DO NOT EDIT) Script process Start Time ##########
StartTime = time.time()
StartDateTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
Current_Dir = os.getcwd()    # Get this script current directory


                        ########## (DO NOT EDIT) LOCATION OF INPUTS, FOLDERS & IMAGES ##########
# Train Data Location    
Train_Facial_Expression = os.path.join(Current_Dir,'Train_DATA\Train_DATA_Facial_Expressions.xlsx')  
# Test Data Location
Test_Facial = [os.path.join(Current_Dir,'DATA',X) for X in os.listdir(os.path.join(Current_Dir,'DATA'))]   
# Frontal Face Detection
Frontal_Face_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_frontalface_default.xml')
# Profile Face Detection
Profile_Face_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_profileface.xml')
# Eyes Detection
Eyes_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_eye.xml')
    

                        ########## (DO NOT EDIT) FUNCTIONS ##########
ImageArray, Emotion_No, EmotionList = Read_TrainDATA_Facial_Expression()
score, model = Machine_Learning_Model(ImageArray)
Facial_Expression_Recognition(Test_Facial, Frontal_Face_Detection_XML, Profile_Face_Detection_XML, Eyes_Detection_XML)
Duration_Calculation()





Start: 2019-12-27 14:39:12 	    Loss: 0.716
End  : 2019-12-27 14:39:45 	Accuracy: 91.52 %
Runtime: 0 Hrs 0 Min 32 Sec
